## Primary vs Primary + Blooming

In this notebook all the images with the label primary (but not blooming) will be put against images with the label primary and blooming. 

### Imports

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau

import numpy as np
import cPickle as pickle
import pandas as pd
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 Ti (0000:09:00.0)


### Loading training data and corresponding labels. 

Images = ['image_name', 'primary', 'blooming']

In [2]:
batch_size = 64
size_img = 64
imgs = (size_img,size_img)
#train_csv = data.get_class_data(train=True, label='primary')

#label is blooming

train_csv = np.asarray(data.get_class_data(train=True, label='primary')) #everything in onehot
val_csv =  np.asarray(data.get_class_data(train=False, label='primary')) #everything in onehot

train_data = []
for i in train_csv:
    if i[1] == 1:
        appending = [i[0], int(i[1]), int(i[2])]
        train_data.append(appending)
train_csv = np.asarray(train_data)

val_data = []
for i in val_csv:
    if i[1] == 1:
        appending = [i[0], int(i[1]), int(i[2])]
        val_data.append(appending)
val_csv = np.asarray(val_data)


train_data = train_csv[:,0] #image names training
val_data = val_csv[:,0] #image names validation


train_labels = train_csv[:,1:] 
val_labels = val_csv[:,1:]

print train_labels.shape
print val_labels.shape

#train_labels = train_csv[:,11:12] #primary
#val_labels = val_csv[:,2:3]


(26284, 2)
(11229, 2)


In [3]:
dim = len(train_labels)
dimlabel = np.zeros((dim, 2))

for i, j in zip(dimlabel, train_labels[:,1]):
    i[0] = 1
    i[1] = j[0]

train_labels = dimlabel

dim = len(val_labels)
dimlabel = np.zeros((dim, 2))
for i, j in zip(dimlabel, val_labels[:,1]):
    i[0] = 1
    i[1] = j[0]

val_labels = dimlabel

In [5]:
print train_labels.shape
print val_labels.shape
print val_data.shape
print train_data.shape

print train_labels[1]
print val_labels[1]
print train_data[1]
print val_data[1]


(26284, 2)
(11229, 2)
(11229,)
(26284,)
[ 1.  0.]
[ 1.  0.]
train_8337
train_19035


#### Generators

In [ ]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels, batch_size=batch_size, img_size=imgs, balance_batches=True, augmentation=True, hflip=True, vflip=True, shift_x=5, shift_y=5, rot_range=10)
val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=batch_size, img_size=imgs, balance_batches=True)

In [ ]:
a, b = train_gen.next()


### Network Structure

In [ ]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(3, 64, 64)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

### Training Network

In [ ]:
steps = len(train_data)/batch_size
steps_val = len(val_data)/batch_size
print steps
print steps_val


csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='/home/pieter/projects/MLIP-Brainforest/bloomingtest/OWN/model.{epoch:02d}-{val_loss:.2f}.hdf5',
                             verbose=1, save_best_only=True)

model.fit_generator(train_gen, steps_per_epoch=steps,
                    epochs=10, verbose=1,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, validation_steps=steps_val)